In [1]:
from prody import *
import numpy as np
import csv
import pandas as pd
import sys, os
from glob import glob
# Add the main the parent directory
sys.path.append(r'..')

In [2]:
# Protein name
prot_name = 'fxa'
# Protein sequence: P28482 (ERK2_HUMAN)
uniprot_id = "P00742"

In [3]:

# Open the protein data table
path_df_csv = glob(os.path.join('..', 'data', 
                        F'TABLA_MTDATA_{prot_name.upper()}_*_crys.json'))
df_prot = pd.read_json(path_df_csv[0])
df_prot.set_index('PDB_ID', inplace=True)


In [4]:
# How many structures have a HETATM?
df_prot_ligs = df_prot[df_prot.NumLigs > 0]
print(F'{len(df_prot_ligs)} crystals {len(df_prot)} have at least one HETATM')

# How many ligands are there?
import itertools
# List_ name all ligands
lig_full_list = list( itertools.chain.from_iterable( df_prot.NameLigs ) )
print(F'There are {len(lig_full_list)} HETATM molecules/atoms')

lig_all_list = list(set(lig_full_list))
print(F'There are {len(lig_all_list)} DIFFERENT molecules/atoms.')

134 crystals 136 have at least one HETATM
There are 314 HETATM molecules/atoms
There are 147 DIFFERENT molecules/atoms.


## Moléculas en la cavidad catalítica
Ahora procedemos a identificar qué moléculas se encuentran en el sitio activo de la proteína, y si éstas tienen un peso molecular suficcientemente grande como para ser diferenciadas de posibles moléculas del solvente.

### Estructura de referencia
Tomamos como referencia alguna estructura del DataFrame con ligandos:

De acuerdo a BindingDatabase, el mejor ligando (y la estructura con él es: http://www.rcsb.org/pdb/results/results.do?tabtoshow=Current&qrid=EA4EE133

2P3T y ligando 993

In [5]:
from modules.get_cocristalized_ligands import PocketResidues

_ColormakerRegistry()

In [86]:
prot_main_dir = '../../FILES/CRYSTALS/PROT_FXA_PREPARED/PREP_ALL_ENSAMBLE'
raw_ligs_dir = '../../FILES/CRYSTALS/LIGS_FXA/RAW'

pdb_id = '1ezq'
ligand_resname = 'RPR'

ref_structure = PocketResidues(pdb_id, ligand_resname, prot_main_dir, raw_ligs_dir)
# Get the pocket redidues
pocket_residues = ref_structure.get_pocket_residues_as_str(cutoff=5)
pocket_residues

@> 3783 atoms and 1 coordinate set(s) were parsed in 0.08s.
@> 144 atoms and 1 coordinate set(s) were parsed in 0.00s.


'131 134 161 167 178 179 180 181 182 183 184 202 203 204 205 206 207 208 209 82 83 84 210 214 215 216 217'

In [87]:
# Visualize the pocket residues
view = ref_structure.visualize_pocket(cutoff=5)
view

NGLWidget()

In [85]:
# pdb_id = '3ens'
# ligand_resname = 'ENS'

# ref_structure = PocketResidues(pdb_id, ligand_resname, prot_main_dir, raw_ligs_dir)
# # Get the pocket redidues
# pocket_residues = ref_structure.get_pocket_residues_as_str(cutoff=5)
# view = ref_structure.visualize_pocket(cutoff=5)
# view

In [10]:
sdfs

NameError: name 'sdfs' is not defined

In [ ]:
''.join(str(pocket_list)).replace('[', '').replace(']', '')

In [ ]:
# Just for visualization

import nglview as nv

# Select pocket residues
str_residues = ''.join(str(pocket_list)).replace('[', '').replace(']', '').replace(',', '')
# Get list of atoms
pocket_atoms = prot.select('resid ' + str_residues).getIndices()
# Load ligand as nglview object
ligand_nv = nv.ProdyStructure(ligand)

view = nv.show_prody(prot)
view.clear_representations()
view.add_representation('cartoon', selection='protein', color='white')
view.add_licorice(selection=pocket_atoms, color='red')
view.add_cartoon(selection=pocket_atoms, color='red')
view.add_structure(ligand_nv)
view

### uso de la estructura de referencia

In [ ]:
# Cargamos la estructura PDB usando prody
pdbid_ref_name = '1ezq'
ligand_name = 'RPR'
cutoff = 6
structure = parsePDB(F'../../FILES/CRYSTALS/PDB_{prot_name.upper()}_RAW_files/{pdbid_ref_name}.pdb.gz')

# Selecion de los residuos a 8 agstroms del ligando en la cavidad
selected_residues = \
structure.select(F'within {cutoff} of resname {ligand_name} and name CA and chain A')

ligand = structure.select(F'resname {ligand_name}')
# Obtenemos la secuencia de los residuos
residues_list = selected_residues.getResnums()


# Convertimos la lista de resiudos a un string
residues_list_str = " ".join(residues_list.astype(str))

print(residues_list_str)

### Visualización del sitio activo EN LA ESTRUCTURA CRISTAL (No en le modelo preparado)

In [ ]:
# Seleccionamos todos los átomos de los residuos en selected_residues
pocket_atoms = structure.select(F'resnum {residues_list_str}')
# Obtenemos los índices de los átomos
pocket_atoms_index = pocket_atoms.getIndices()

In [ ]:
import nglview as nv
view = nv.show_prody(structure)
view.clear_representations()
view.add_representation('cartoon', selection='protein', color='white')
view.add_ball_and_stick(selection = ligand_name, color='orange', linewidth=3.)
view.add_licorice(selection = pocket_atoms_index, color='red')
view.add_cartoon(selection = pocket_atoms_index, color='red')
view

In [ ]:
# Cálculo del centro total de la proteína de referencia
centro_strc_total = calcCenter(structure).round(3)

# Definimos los átomos del pocket
pocket = structure.select("resnum " + residues_list_str)
pocket_center = calcCenter(pocket)

# Solo para comparar:
print("Centro de la estructura completa:", centro_strc_total)
print("Centro considerando todos los átomos de los residuos del pocket:", pocket_center)

In [ ]:
view_2 = nv.show_prody(structure)
view_2.shape.add_sphere( pocket_center.tolist(), [0, 0, 0], 5)
view_2.add_licorice(selection = pocket.getIndices(), color='magenta')
view_2

Cualquier molécula que tenga un átomo en contacto con la esfera, será considerada como ligando unido al sitio activo.

## Determinación de los ligandos
Ahora, procedemos a utilizar la estructura de referencia modelada con Modeller. Hay que recordar que todas las estructuras modeladas fueron alineadas previamente tomando en cuenta la estructura secundaria de la proteína, y que el alineamiento incluyó la reposición relativa de los ligandos.


### Pasos y función para obtener los ligandos

In [150]:
# Definimos los directorios de trabajo
main_dir = '../../FILES/CRYSTALS'
RAW_LIG_DIR = F'{main_dir}/LIGS_{prot_name.upper()}/RAW/'
PROT_CHAINS_DIR = F'{main_dir}/PROT_{prot_name.upper()}_PREPARED/PREP_ALL_ENSAMBLE/'

# Directorio de salida (lo creamos si no existe):
PK_LIGS_DIR = F'{main_dir}/LIGS_{prot_name.upper()}/POCKET_LIGS/'
import os
if not os.path.exists(PK_LIGS_DIR):
    os.makedirs(PK_LIGS_DIR)

In [153]:
# Ligando
pdb_id = '2vvc'
raw_lig_dir = RAW_LIG_DIR
lig_file = glob(raw_lig_dir + pdb_id  + "*.pdb")[0]
lig = parsePDB(lig_file)

@> 645 atoms and 1 coordinate set(s) were parsed in 0.01s.


In [193]:
from glob import glob

def get_pocket_ligand(pdb_id,
                       pocket_residues,
                       cutoff = 3,
                       raw_lig_dir = RAW_LIG_DIR,
                       prot_chain_dir = PROT_CHAINS_DIR,
                       pk_ligs_dir = PK_LIGS_DIR,
                       min_weight = 97, # mw de sulfato  + 1
                       write_files = True):
    
    # 1) Se carga cargan las moléculas HETATM usando Prody
    try:
        lig_file = glob(raw_lig_dir + pdb_id  + "*.pdb")[0]
        lig = parsePDB(lig_file)
    except FileNotFoundError as e:
        print("The protein", pdb_id, "have NO LIGAND.")
        return(None, None)
    
    # 2) Se carga la proteína y se seleccionan los residuos que definen el pocket
    try:
        prot_file = glob(prot_chain_dir + pdb_id + "*.pdb")[0]
        protein = parsePDB( prot_file )
    except FileNotFoundError as e:
        print("Protein file not found:", pdb_id)
        return(None, None)
    
    protein_pocket = protein.select("resid " + pocket_residues)
    
    # Se seleccionan los ligandos (RESIDUOS no protéicos) que estén a no más de cutoff A
    # de cualquier átomo de los residuos del pocket de la proteína
    lig_sel = lig.select('within ' + str(cutoff) + ' of inhibitor', inhibitor = protein_pocket)
    if lig_sel is None:
        print("La proteina", pdb_id, "no tiene ligando en el pocket.")
        return(None, None)
    
    # Se obtine la lista de moléculas que cumplen el criterio anterior
    inhibidor_list = np.unique( lig_sel.getResnames() )
    print(inhibidor_list)
    
    # Calcula el centro geométrico del pocket de la proteína
    prot_pocket_center = calcCenter(protein_pocket)
    
    # Puede que haya más de una molécula en el pocket 
    # (debido a presencia de iones, cosolvente o residuos modificados)
    # Se itera entre cada molecula del ligando, se calcula su masa y se mantiene el ligando con mayor masa 
    distance = 10000
    nearest_chain = ''
    nearest_resnum = ''
    nearest_resname = ''
    current_mass = 0
    true_lig = None

    for resname in inhibidor_list:
            mol = lig.select("resname " + resname)
            
            # Itera entre todos las moléculas definidas por un resname, un resnum y una cadena
            mol_chains = np.unique(mol.getChids())
            for chain in mol_chains:
                mol_resnums = np.unique(mol.select('chain ' + chain).getResnums())
                for resnum in mol_resnums:
                    new_mol = mol.select('chain ' + chain + \
                                  ' and resnum ' + str(resnum))
                    new_mass = new_mol.getMasses().sum()
                    
                    #print(new_mass, resname, chain)
                    if new_mass > current_mass:
                    
                    
                        dist_new_mol = calcDistance(prot_pocket_center, calcCenter(new_mol))
#                     #print(chain, resnum, resname, dist_new_mol)
#                     if dist_new_mol < distance:
                        nearest_chain = chain
                        nearest_resnum = str(resnum)
                        nearest_resname = resname
                        distance = dist_new_mol
                        current_mass = new_mass
                        
    true_lig = lig.select('chain ' + nearest_chain + \
                          ' and resnum ' + nearest_resnum + \
                          ' and resname ' + nearest_resname)
    lig_mass = true_lig.getMasses().sum()

            
    # Se guarda el ligando sólo si su masa es igual o mayor a la de un sulfato
    # si el ligando aparece 2 veces cerca del pocket, se elige la molécula más cercana al centro
    if true_lig != "" and lig_mass > min_weight: # Si el ligando existe y su masa fue mayor 96
        # Se extrae el nombre del ligando
        name_lig = np.unique( true_lig.getResnames() )[0]

        # Se combinan proteína y ligando en un solo objeto AtomGroup, de tal manera que
        # sea posible superponer las estructuras de la proteína de acuerdo a los residuos de su pocket
        # y esto afcete la psoición del ligando
        complejo_PL = protein + true_lig.toAtomGroup()
        if write_files:
            # Se guardan ligando y proteína alineados según la estructura de referencia
            print(F'Proteína {pdb_id}: ligando {name_lig} guardado.')
            writePDB( pk_ligs_dir + "/" + pdb_id + "_" + name_lig + "_LIG.pdb", 
                      complejo_PL.select("resname " + name_lig) )
        return(name_lig, lig_mass)
    else: 
        print(F"La proteina {pdb_id} NO TIENE LIGANDO.")
        return(None, None)

In [195]:
from modules.get_cocristalized_ligands import get_pocket_ligand

# Se agregan dos columnas al dataframe
df_prot["Inhib"] = ""
df_prot["Inhib_mass"] = np.nan

# Se obtienen los ligandos
for pdb_id in df_prot.index:
    inhibidor = get_pocket_ligand(pdb_id, pocket_residues=pocket_residues,
                                  raw_lig_dir= RAW_LIG_DIR,
                                  prot_chain_dir=PROT_CHAINS_DIR,
                                  pk_ligs_dir=PK_LIGS_DIR,
                                  write_files=True, cutoff=3.5, min_weight = 97)
    
    df_prot.at[str(pdb_id), 'Inhib'] = inhibidor[0]
    df_prot.at[pdb_id, 'Inhib_mass'] = inhibidor[1]

@> 1338 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 3783 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> 144 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 3783 atoms and 1 coordinate set(s) were parsed in 0.04s.


['HOH']
La proteina 1c5m NO TIENE LIGANDO.
['HOH' 'RPR']
Proteína 1ezq: ligando RPR guardado.


@> 202 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 3783 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 177 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 3783 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 40 atoms and 1 coordinate set(s) were parsed in 0.00s.


['815' 'HOH']
Proteína 1f0r: ligando 815 guardado.
['HOH' 'PR2']
Proteína 1f0s: ligando PR2 guardado.


@> 3783 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> 239 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 3783 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> 253 atoms and 1 coordinate set(s) were parsed in 0.01s.


...

['4O4' 'HOH']
Proteína 4zh8: ligando 4O4 guardado.
['4O5' 'HOH' 'MG']
Proteína 4zha: ligando 4O5 guardado.


@> 3783 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 543 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 3783 atoms and 1 coordinate set(s) were parsed in 0.06s.
@> 506 atoms and 1 coordinate set(s) were parsed in 0.00s.


['6PK' 'CA' 'HOH']
Proteína 5k0h: ligando 6PK guardado.
['HOH' 'NA']
La proteina 5voe NO TIENE LIGANDO.


@> 3783 atoms and 1 coordinate set(s) were parsed in 0.07s.


['HOH' 'NA' 'RIV']
Proteína 5vof: ligando RIV guardado.
